<a href="https://colab.research.google.com/github/davidcechak/EmbedDTI/blob/master/EmbedDTI_playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!git clone https://github.com/davidcechak/EmbedDTI.git

fatal: destination path 'EmbedDTI' already exists and is not an empty directory.


In [25]:
!ls

data  EmbedDTI	sample_data


In [26]:
!cd EmbedDTI

In [27]:
!pip install rdkit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [28]:
!pip install torch_geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [29]:
# FIX torch-sparse bug https://stackoverflow.com/a/71764992
import torch 
print(torch.__version__)
!pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster --yes
!pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

1.12.0+cu113
Found existing installation: torch-scatter 2.0.9
Uninstalling torch-scatter-2.0.9:
  Successfully uninstalled torch-scatter-2.0.9
Found existing installation: torch-sparse 0.6.14
Uninstalling torch-sparse-0.6.14:
  Successfully uninstalled torch-sparse-0.6.14
Found existing installation: torch-geometric 2.0.5
Uninstalling torch-geometric-2.0.5:
  Successfully uninstalled torch-geometric-2.0.5
Found existing installation: torch-cluster 1.6.0
Uninstalling torch-cluster-1.6.0:
  Successfully uninstalled torch-cluster-1.6.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.12.0+cu113.html
  Using cached https://data.pyg.org/whl/torch-1.12.0%2Bcu113/torch_scatter-2.0.9-cp37-cp37m-linux_x86_64.whl (7.9 MB)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.12.0+cu113.html
  Using cached https://data.pyg.org/whl/torch-1.12.0%2Bcu113/torch_sparse-0.6.14-cp37-cp37m-linux_x86_64.whl (3.5 MB)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.12.0+cu113.html
  Using cached https://data.pyg.org/whl/torch-1.12.0%2Bcu113/torch_cluster-1.6.0-cp37-cp37m-linux_x86_64.whl (2.4 MB)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/pyg-team/pytorch_geometric.git to /tmp/pip-req-build-bylbs52m
  Running command git clone -q https://github.com/pyg-team/pytorch_geometric.git /tmp/pip-req-build-bylbs52m
  Created wheel for torch-geometric: filename=torch_geometric-2.0.5-py3-none-any.whl size=679050 sha256=9cf576864d4420dff7934b9fd62267a38d009d1ef22f1c546b7ac51daa2be87a
  Stored in directory: /tmp/pip-ephem-wheel-cache-udrtjjga/wheels/85/c9/07/7936efecad79b906348a7e9fb644d914160544efa9aa7f4b2b
Successfully built torch-geometric


In [30]:
#coding=utf-8
import pandas as pd
import numpy as np
import os
import json,pickle
from collections import OrderedDict
from rdkit import Chem
from rdkit.Chem import MolFromSmiles
import networkx as nx
from scipy.sparse import csr_matrix
# imports TestbedDataset, rmse, mse, pearson, spearman, ci
from EmbedDTI.utils import *
from collections import defaultdict
from rdkit.Chem.EnumerateStereoisomers import EnumerateStereoisomers, StereoEnumerationOptions
from rdkit.Chem import AllChem


def read_data(filename):
    df = pd.read_csv('EmbedDTI/data/'+filename)
    drugs,proteins,Y = list(df['compound_iso_smiles']),list(df['target_sequence']),list(df['affinity'])

    return drugs,proteins,Y


# read files(train and test)
davis_train_drugs,davis_train_proteins,davis_train_Y = read_data("davis_train.csv")
davis_test_drugs,davis_test_proteins,davis_test_Y = read_data("davis_test.csv")
drugs = davis_train_drugs+davis_test_drugs
print(len(drugs)) # 30056


# 处理蛋白质序列，构建蛋白质字典
# Process protein sequences and build protein dictionaries
seq_voc = "ABCDEFGHIKLMNOPQRSTUVWXYZ"
seq_dict = {v:(i+1) for i,v in enumerate(seq_voc)}
seq_dict_len = len(seq_dict)
max_seq_len = 1000
# print(seq_dict)


def seq_cat(prot):
    x = np.zeros(max_seq_len)
    for i, ch in enumerate(prot[:max_seq_len]): 
        x[i] = seq_dict[ch]
    return x  

train_prots = [seq_cat(t) for t in davis_train_proteins]
test_prots = [seq_cat(t) for t in davis_test_proteins]
print(len(train_prots))  # 25046
print(len(test_prots))   # 5010


# 处理SMILES序列，转化为原子的图结构
# Process SMILES sequences into graph structures of atoms
def atom_features(atom):
    return np.array(one_of_k_encoding_unk(atom.GetSymbol(),['C', 'N', 'O', 'S', 'F', 'Si', 'P', 'Cl', 'Br', 'Mg', 'Na','Ca', 'Fe', 'As', 'Al', 'I', 'B', 'V', 'K', 'Tl', 'Yb','Sb', 'Sn', 'Ag', 'Pd', 'Co', 'Se', 'Ti', 'Zn', 'H','Li', 'Ge', 'Cu', 'Au', 'Ni', 'Cd', 'In', 'Mn', 'Zr','Cr', 'Pt', 'Hg', 'Pb', 'Unknown']) +
                    one_of_k_encoding(atom.GetDegree(), [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]) +
                    one_of_k_encoding_unk(atom.GetTotalNumHs(), [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]) +
                    one_of_k_encoding_unk(atom.GetImplicitValence(), [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]) + # 返回原子上隐式Hs的数量
                    one_of_k_encoding_unk(atom.GetTotalValence(),[0,1,2,3,4,5,6,7,8,9,10]) +
                    one_of_k_encoding_unk(atom.GetFormalCharge(),[0,1,2,3,4,5,6,7,8,9,10]) +
                    [atom.GetIsAromatic()] +
                    [atom.IsInRing()]
                    )

def one_of_k_encoding(x, allowable_set):
    if x not in allowable_set:
        raise Exception("input {0} not in allowable set{1}:".format(x, allowable_set))
    return list(map(lambda s: x == s, allowable_set))

def one_of_k_encoding_unk(x, allowable_set):
    """Maps inputs not in the allowable set to the last element."""
    if x not in allowable_set:
        x = allowable_set[-1]
    return list(map(lambda s: x == s, allowable_set))


def smile_to_graph(smile):
    mol = Chem.MolFromSmiles(smile)
    
    c_size = mol.GetNumAtoms()  # 原子数 Atomic number
    
    features = [] 
    for atom in mol.GetAtoms():
        feature = atom_features(atom)
        features.append( feature / sum(feature) )

    edges = []
    for bond in mol.GetBonds():
        edges.append([bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()])
    # 构建有向图 Building a directed graph
    g = nx.Graph(edges).to_directed()
    edge_index = []
    for e1, e2 in g.edges:
        edge_index.append([e1, e2])
    return c_size, features, edge_index


compounds = []
for smile in drugs:
    lg = Chem.MolToSmiles(Chem.MolFromSmiles(smile),isomericSmiles=True)
    compounds.append(lg)
print("原始SMILES数量：Original SMILES quantity", len(compounds))  # 30056 
smile_graph = {}
compound_smiles = set(compounds)
print("不重复的SMILES数量: Number of unique SMILES", len(compound_smiles))   # 68
for smile in compound_smiles:
    g = smile_to_graph(smile) # c_size, features, edge_index
    smile_graph[smile] = g
print("simile_graph长度:", len(smile_graph))  # 68



# # 处理SMILES序列，转化为基团的图结构 Process SMILES sequences into graph structures of groups

MST_MAX_WEIGHT = 100 

def get_mol(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None: 
        return None
    Chem.Kekulize(mol)
    return mol

# 构建邻接矩阵和特征矩阵
# Build Adjacency Matrix and Feature Matrix
# the "idx" argument is not used in the cluster_graph function
# originally: def cluster_graph(mol,idx):
def cluster_graph(mol,idx=None):
	n_atoms = mol.GetNumAtoms()
	# if n_atoms == 1: #special case
 #    	return [[0]], []
	cliques = []
	for bond in mol.GetBonds():
		a1 = bond.GetBeginAtom().GetIdx()
		a2 = bond.GetEndAtom().GetIdx()
		if not bond.IsInRing():
			cliques.append([a1,a2])

	ssr = [list(x) for x in Chem.GetSymmSSSR(mol)]
	cliques.extend(ssr)

	# nei_list为原子属于哪个基团
	nei_list = [[] for i in range(n_atoms)]
	for i in range(len(cliques)):
	    for atom in cliques[i]:
	        nei_list[atom].append(i)

	#Merge Rings with intersection > 2 atoms
	for i in range(len(cliques)):
		if len(cliques[i]) <= 2: continue
		for atom in cliques[i]:
			for j in nei_list[atom]:
				if i >= j or len(cliques[j]) <= 2: 
					continue
				inter = set(cliques[i]) & set(cliques[j])
				if len(inter) > 2:
					cliques[i].extend(cliques[j])
					cliques[i] = list(set(cliques[i]))
					cliques[j] = []
    
	cliques = [c for c in cliques if len(c) > 0]
	nei_list = [[] for i in range(n_atoms)]
	for i in range(len(cliques)):
		for atom in cliques[i]:
			nei_list[atom].append(i)

	# Build edges and add singleton cliques
	edges = defaultdict(int)
	for atom in range(n_atoms):
		if len(nei_list[atom]) <= 1: 
			continue
		cnei = nei_list[atom]
		bonds = [c for c in cnei if len(cliques[c]) == 2]
		rings = [c for c in cnei if len(cliques[c]) > 4]
		
		for i in range(len(cnei)):
			for j in range(i + 1, len(cnei)):
				c1,c2 = cnei[i],cnei[j]
				inter = set(cliques[c1]) & set(cliques[c2])
				if edges[(c1,c2)] < len(inter):
					edges[(c1,c2)] = len(inter) # cnei[i] < cnei[j] by construction
					edges[(c2,c1)] = len(inter)

	edges = [u + (MST_MAX_WEIGHT-v,) for u,v in edges.items()]
	row,col,data = zip(*edges)
	data = list(data)
	for i in range(len(data)):
		data[i] = 1
	data = tuple(data)
	n_clique = len(cliques)
	clique_graph = csr_matrix( (data,(row,col)), shape=(n_clique,n_clique) )
	edges = [[row[i],col[i]] for i in range(len(row))]
	
	return cliques, edges



def clique_features(clique,edges,clique_idx,smile):
    NumAtoms = len(clique) # 基团中除去氢原子的原子数
    NumEdges = 0  # 与基团所连的边数
    for edge in edges:
        if clique_idx == edge[0] or clique_idx == edge[1]:
            NumEdges += 1
    mol = Chem.MolFromSmiles(smile)
    NumHs = 0 # 基团中氢原子的个数
    for atom in mol.GetAtoms():
        if atom.GetAtomMapNum() in clique:
            NumHs += atom.GetTotalNumHs()
    # 基团中是否包含环
    IsRing = 0
    if len(clique) > 2:
        IsRing = 1
    # 基团中是否有键
    IsBond = 0
    if len(clique) == 2:
        IsBond = 1

    return np.array(one_of_k_encoding_unk(NumAtoms,[0,1,2,3,4,5,6,7,8,9,10]) + 
        one_of_k_encoding_unk(NumEdges,[0,1,2,3,4,5,6,7,8,9,10]) + 
        one_of_k_encoding_unk(NumHs,[0,1,2,3,4,5,6,7,8,9,10]) + 
        [IsRing] + 
        [IsBond]
        )

# Adjacency matrix, three-dimensional matrix, each graph is a two-dimensional matrix
cluster = [] # 邻接矩阵, 三维矩阵，每一张图是一个二维矩阵 
# Connecting edge matrix, three-dimensional matrix, each graph is a two-dimensional matrix
edges = []  # 连接边矩阵, 三维矩阵，每一张图是一个二维矩阵

for i,smile in enumerate(compound_smiles):
    try:
        mol = get_mol(smile)
        clique,edge = cluster_graph(mol,i)
        cluster.append(clique)
        edges.append(edge)
    except:
        print('Error:',i)

print(len(cluster))  # 68
print(len(edges))   # 68

clique_graph = {}
for i,smile in enumerate(compound_smiles):
    c_features = []
    for idx in range(len(cluster[i])):
        cq_features = clique_features(cluster[i][idx],edges[i],idx,smile)
        c_features.append( cq_features / sum(cq_features) )
    clique_size = len(cluster[i])
    graph = (clique_size, c_features, edges[i])
    clique_graph[smile] = graph

print("clique_graph长度:", len(clique_graph))  # 68

train_drugs, train_prots,  train_Y = np.asarray(davis_train_drugs), np.asarray(train_prots), np.asarray(davis_train_Y)
test_drugs, test_prots,  test_Y = np.asarray(davis_test_drugs), np.asarray(test_prots), np.asarray(davis_test_Y)

train_data = TestbedDataset(root='data', dataset='davis_train', xd=train_drugs, xt=train_prots, y=train_Y,smile_graph=smile_graph,clique_graph=clique_graph)
test_data = TestbedDataset(root='data', dataset='davis_test', xd=test_drugs, xt=test_prots, y=test_Y,smile_graph=smile_graph,clique_graph=clique_graph)







30056
25046
5010
原始SMILES数量：Original SMILES quantity 30056
不重复的SMILES数量: Number of unique SMILES 68
simile_graph长度: 68
68
68
clique_graph长度: 68
Pre-processed data found: data/processed/davis_train_atom.pt and data/processed/davis_train_clique.pt, loading ...
Pre-processed data found: data/processed/davis_test_atom.pt and data/processed/davis_test_clique.pt, loading ...


In [ ]:
print(train_data)
print(train_data.data_list_clique)
print(train_data.data_list_atom)
print(train_data.dataset)


TestbedDataset()
Data(x=[390782, 35], edge_index=[2, 821712], y=[25046], target=[25046, 1000], c_size=[25046])
Data(x=[803206, 101], edge_index=[2, 1776072], y=[25046], target=[25046, 1000], c_size=[25046])
davis_train


In [66]:
def embedDTI_smile_preprocess(smile_to_process, test_MolToSmiles):
    # atom representation
    if (test_MolToSmiles):
        smile_to_process = Chem.MolToSmiles(Chem.MolFromSmiles(smile_to_process),isomericSmiles=True)
    atom_result = smile_to_graph(smile_to_process)

    print(type(atom_result))
    print(np.array(atom_result[0]))
    print(np.array(atom_result[1]).shape)
    print(np.array(atom_result[2]).shape)
    print('__________________')


    # substructure representation
    our_mol = get_mol(smile_to_process)
    our_clique, our_edge = cluster_graph(our_mol)

    print(our_mol)
    print(our_clique)
    print(our_edge)

    c_features = []
    for idx in range(len(our_clique)):
        # TODO check if it has to be canonical SMILES or not
        cq_features = clique_features(our_clique[idx],our_edge,idx,smile_to_process)
        c_features.append( cq_features / sum(cq_features) )
    clique_size = len(our_clique)
    graph = (clique_size, c_features, our_edge)

    return atom_result, graph


In [67]:
# our_smile = 'CN(C)CCOC(C1=CC=CC=C1)C1=CC=CC=C1'
for i in [0,10,11,32,4,5]:
  our_smile = drugs[i]
  abc, deef = embedDTI_smile_preprocess(our_smile, True)
  abcX, deefX = embedDTI_smile_preprocess(our_smile, False)


<class 'tuple'>
25
(25, 101)
(54, 2)
__________________
[[0, 1], [1, 2], [2, 3], [1, 9], [13, 14], [14, 15], [15, 16], [15, 17], [18, 19], [23, 24], [4, 5, 6, 7, 8, 3], [10, 9, 13, 12, 11], [18, 20, 21, 22, 23, 17]]
[[0, 1], [1, 0], [0, 3], [3, 0], [1, 3], [3, 1], [1, 2], [2, 1], [2, 10], [10, 2], [3, 11], [11, 3], [4, 11], [11, 4], [4, 5], [5, 4], [5, 6], [6, 5], [5, 7], [7, 5], [6, 7], [7, 6], [7, 12], [12, 7], [8, 12], [12, 8], [9, 12], [12, 9]]
<class 'tuple'>
25
(25, 101)
(54, 2)
__________________
[[0, 1], [1, 2], [2, 3], [1, 9], [13, 14], [14, 15], [15, 16], [15, 17], [18, 19], [23, 24], [4, 5, 6, 7, 8, 3], [10, 9, 13, 12, 11], [18, 20, 21, 22, 23, 17]]
[[0, 1], [1, 0], [0, 3], [3, 0], [1, 3], [3, 1], [1, 2], [2, 1], [2, 10], [10, 2], [3, 11], [11, 3], [4, 11], [11, 4], [4, 5], [5, 4], [5, 6], [6, 5], [5, 7], [7, 5], [6, 7], [7, 6], [7, 12], [12, 7], [8, 12], [12, 8], [9, 12], [12, 9]]
<class 'tuple'>
19
(19, 101)
(42, 2)
__________________
[[0, 1], [4, 5], [5, 6], [5, 7], [11, 

In [31]:
import numpy as np

our_smile = 'CN(C)CCOC(C1=CC=CC=C1)C1=CC=CC=C1'

# ? are isomericSmiles necessery or does Dimitris handles this?
# do we want it to canonical SMILES or not?
# our_smile = Chem.MolToSmiles(Chem.MolFromSmiles(our_smile),isomericSmiles=True)
atom_result = smile_to_graph(our_smile)
print(type(atom_result))
print(np.array(atom_result[0]))
print(np.array(atom_result[1]).shape)
print(np.array(atom_result[2]).shape)

<class 'tuple'>
19
(19, 101)
(40, 2)


In [34]:
our_mol = get_mol(our_smile)
our_clique, our_edge = cluster_graph(our_mol)

print(our_mol)
print(our_clique)
print(our_edge)


[[0, 1], [1, 2], [1, 3], [3, 4], [4, 5], [5, 6], [6, 7], [6, 13], [8, 9, 10, 11, 12, 7], [14, 15, 16, 17, 18, 13]]
[[0, 1], [1, 0], [0, 2], [2, 0], [1, 2], [2, 1], [2, 3], [3, 2], [3, 4], [4, 3], [4, 5], [5, 4], [5, 6], [6, 5], [5, 7], [7, 5], [6, 7], [7, 6], [6, 8], [8, 6], [7, 9], [9, 7]]


In [ ]:
cluster = [our_clique]
edges = [our_edge]
i = 0

c_features = []
for idx in range(len(cluster[i])):
    # TODO check if it has to be canonical SMILES or not
    cq_features = clique_features(cluster[i][idx],edges[i],idx,our_smile)
    c_features.append( cq_features / sum(cq_features) )
clique_size = len(cluster[i])
graph = (clique_size, c_features, edges[i])

In [49]:
print(graph[0])
print(np.array(graph[1]).shape)
print(np.array(graph[2]).shape)

10
(10, 35)
(46, 2)


In [ ]:
# Adjacency matrix, three-dimensional matrix, each graph is a two-dimensional matrix
cluster = [] # 邻接矩阵, 三维矩阵，每一张图是一个二维矩阵 
# Connecting edge matrix, three-dimensional matrix, each graph is a two-dimensional matrix
edges = []  # 连接边矩阵, 三维矩阵，每一张图是一个二维矩阵

for smile in compound_smiles:
    try:
        mol = get_mol(smile)
        clique,edge = cluster_graph(mol)
        cluster.append(clique)
        edges.append(edge)
    except:
        print('Error:')

print(len(cluster))  # 68
print(len(edges))   # 68

clique_graph = {}
for i,smile in enumerate(compound_smiles):
    c_features = []
    for idx in range(len(cluster[i])):
        cq_features = clique_features(cluster[i][idx],edges[i],idx,smile)
        c_features.append( cq_features / sum(cq_features) )
    clique_size = len(cluster[i])
    graph = (clique_size, c_features, edges[i])
    clique_graph[smile] = graph

print("clique_graph长度:", len(clique_graph))  # 68

In [ ]:
# # To use the processed TestbedDataset send it to pytorch DataLoader
# from torch_geometric.data import DataLoader

# TRAIN_BATCH_SIZE = 32
# TEST_BATCH_SIZE = 32
# train_loader = DataLoader(train_data, batch_size=TRAIN_BATCH_SIZE, shuffle=True)
# test_loader = DataLoader(test_data, batch_size=TEST_BATCH_SIZE, shuffle=True)

/usr/local/lib/python3.7/dist-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [ ]:
# next(iter(test_loader))

TypeError: ignored

In [ ]:
# %cd EmbedDTI
# !python3 predict_with_davis_orimodel.py 0
# %cd ..


/content/EmbedDTI
Pre-processed data found: data/processed/davis_test_atom.pt and data/processed/davis_test_clique.pt, loading ...
Traceback (most recent call last):
  File "predict_with_davis_orimodel.py", line 42, in <module>
    test_data = TestbedDataset(root='data', dataset='davis_test')
  File "/content/EmbedDTI/utils.py", line 23, in __init__
    self.data_list_atom, self.slices = torch.load(self.processed_paths[0])
  File "/usr/local/lib/python3.7/dist-packages/torch/serialization.py", line 713, in load
    return _legacy_load(opened_file, map_location, pickle_module, **pickle_load_args)
  File "/usr/local/lib/python3.7/dist-packages/torch/serialization.py", line 920, in _legacy_load
    magic_number = pickle_module.load(f, **pickle_load_args)
_pickle.UnpicklingError: invalid load key, 'v'.
/content
